<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/Query_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsquedas WOS+SCI+SCP+PTJ+CTR para UdeA

Búsquedas en bases bibligráficas  
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
de los artículos científicos de la UdeA

La base de datos se creó con:

[WOS_SCI_SCP_PTJ_CTR.ipynb](./WOS_SCI_SCP_PTJ_CTR.ipynb)

In [1]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus fuzzywuzzy[speedup] > /dev/null

## functions

In [139]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)
from venn import draw_venn, generate_colors
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz
import re
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
idc='CÉDULA'

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel/json or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [3]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x

Overwriting drive.cfg


##  Load data bases

In [3]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

In [5]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')
UDEA=drive_files.biblio['WOS'].copy().reset_index(drop=True)

In [6]:
UDEA[UDEA['UDEA_authors']!=''].shape

(12270, 181)

In [7]:
AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [10]:
AU[:1]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación


## Normalizations

In [13]:
UDEA['DI']=UDEA.apply( 
     lambda row: row['DI'] if row['DI'] else row['SCP_DOI'],axis=1 ).str.strip().str.lower()
UDEA['DI']=UDEA.apply( 
     lambda row: row['DI'] if row['DI'] else row['SCI_DI'],axis=1 ).str.strip().str.lower()

Normalize 'AU'

In [14]:
UDEA['AU']=UDEA['AU'].apply(lambda l: l if re.search('\n$',l) 
                 else 
                 '\n'.join(
            [
    re.sub( '([\w\-\s]+)(\s[\w]\.)',r'\1,\2',s ,re.UNICODE   ) 
                 for s in l.split(', ')]+['']
           ).replace('.','') )

In [15]:
UDEA[~UDEA['AU'].str.contains('\n$')].shape

(0, 181)

In [16]:
def fix_NOMBRE_COMPLETOS(l):
    for d in l:
        if not d.get('NOMBRE COMPLETO'):
            d['NOMBRE COMPLETO']=''
            if d.get('NOMBRES'):
                d['NOMBRE COMPLETO']=d['NOMBRE COMPLETO']+d.get('NOMBRES')
            if d.get('PRIMER APELLIDO'):
                d['NOMBRE COMPLETO']=d['NOMBRE COMPLETO']+' '+d.get('PRIMER APELLIDO')
            if d.get('SEGUNDO APELLIDO'):
                d['NOMBRE COMPLETO']=d['NOMBRE COMPLETO']+' '+d.get('SEGUNDO APELLIDO')            
    return l

In [17]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(fix_NOMBRE_COMPLETOS)

In [18]:
dfid=UDEA[['UDEA_nombre','UDEA_cedula']]
dfid=dfid[dfid['UDEA_cedula'].apply(lambda n: n!=0 if isinstance(n,int) else False)].reset_index(drop=True)
dfid=dfid[dfid['UDEA_nombre']!=''].drop_duplicates().reset_index(drop=True)
dfid.shape

(975, 2)

In [19]:
def get_cedula(l,fullnames_with_id=dfid):
    for d in l:
        if not d.get('CÉDULA') and d.get('full_name'):
            try:
                d['CÉDULA']=dfid[ ( dfid['UDEA_nombre']==d.get('full_name')) 
                            & ( dfid['UDEA_cedula']!=0 ) 
                            ].reset_index(
                          drop=True)['UDEA_cedula'].loc[0]
            except KeyError:
                pass
    return l

In [20]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(get_cedula)

In [21]:
def normalize_NAME(row,WOS_NAME,SCI_NAME,SCP_NAME,UDEA_NAME,
                   SCI_sub=('',''),SCP_sub=('',''),UDEA_sub=('',''),
                  DEBUG=False):
    sn=row[WOS_NAME]
    if not sn:
        if SCI_NAME and row[SCI_NAME]:
            sn=re.sub( SCI_sub[0],SCI_sub[1], str( row[SCI_NAME] ) )
            if DEBUG:
                print('SCI: {}:{}'.format(row[SCI_NAME], sn))
        elif SCP_NAME and row[SCP_NAME]:
            sn=re.sub( SCP_sub[0],SCP_sub[1], str( row[SCP_NAME] ) )
        elif UDEA_NAME and row[UDEA_NAME]:
            sn=re.sub( SCP_sub[0],SCP_sub[1], str( row[UDEA_NAME] ) )
            if DEBUG:
                print('UDEA: {}:{}'.format(row[UDEA_NAME], sn))            
    return sn

In [22]:
WOS_NAME ='SN'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_SN',
                                      SCP_NAME ='SCP_ISSN',
                                      UDEA_NAME='UDEA_issn rev',
                                      SCP_sub=( '^([\w]{4})([\w]{4})$',r'\1-\2\n'  ),
                                      UDEA_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [23]:
WOS_NAME ='DI'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_DI',
                                      SCP_NAME ='SCP_DOI',
                                      UDEA_NAME='UDEA_doi')
                      ,axis='columns')

In [24]:
WOS_NAME ='VL'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_VL',
                                      SCP_NAME ='SCP_Volume',
                                      UDEA_NAME='',
                                      SCI_sub=('(\w)$',r'\1\n'),
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [25]:
WOS_NAME ='IS'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_IS',
                                      SCP_NAME ='SCP_Issue',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [26]:
WOS_NAME ='BP'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_BP',
                                      SCP_NAME ='SCP_Page start',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [27]:
WOS_NAME ='EP'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_EP',
                                      SCP_NAME ='SCP_Page end',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [28]:
UDEA['UDEA_idioma']=UDEA['UDEA_idioma'].apply(lambda s: 'Spanish' if s=='ESPA?' else s)
UDEA['UDEA_idioma']=UDEA['UDEA_idioma'].apply(lambda s: 'English' if s=='INGLE' else s)
WOS_NAME ='LA'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_LA',
                                      SCP_NAME ='SCP_Language of Original Document',
                                      UDEA_NAME='UDEA_idioma',
                                      SCI_sub =('\t',r''),
                                      SCP_sub =('(\w)$',r'\1\n'),
                                      UDEA_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [29]:
UDEA.to_json('WOS_SCI_SCP_PTJ_CTR.json.gz')

## Centros

In [393]:
udea=UDEA.copy()

In [394]:
def convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(df,
                            json_column='UDEA_authors',dictionary_key='FACULTAD',sep='; '):
    return df[json_column].apply(lambda l:  
       [d.get(dictionary_key) for d in  l if type(l)==list and d.get(dictionary_key) ]  
               ).apply(
        pd.np.unique
        ).apply(sep.join)    

In [395]:
JSON=True
if not JSON:
    udea['FACULTAD']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='FACULTAD')
    udea['DEPARTAMENTO']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='DEPARTAMENTO')
    udea['GRUPO']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='GRUPO')
    udea['autores_UDEA']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='full_name')

# Prepare columns

In [396]:
def new_Autores(l,nombre='Nombres',tipo='Tipo',identificación='Identificación'):
    dd={}
    ll=[]
    for d in l:
        if d.get('NOMBRE COMPLETO'):
            dd[nombre]=d['NOMBRE COMPLETO']
            dd[tipo]='Profesor vinculado'
        if d.get('CÉDULA'):
            dd[identificación]=d['CÉDULA']
        ll.append(dd)
    return ll

In [397]:
ex.columns

Index(['annio', 'autores', 'clasificacionColciencias', 'doi', 'idioma', 'issn',
       'nombreRevista', 'numero', 'paginas', 'proyectoAsociado', 'semestre',
       'tipoRevista', 'tituloArticulo', 'volumen'],
      dtype='object')

In [398]:
if JSON:
    año='annio'; título='tituloArticulo'; revista='nombreRevista'
    issn='issn'; doi='doi'; volumen='volumen'; número='numero'
    páginas='paginas'; semestre='semestre'; idioma='idioma'
    tipoRevista='tipoRevista';     Colciencias='clasificacionColciencias'
    proyecto='proyectoAsociado'
    autores='autores'
else:
    año='Año';   título='Título del Artículo'; revista='Nombre de la Revista'
    issn='ISSN'; doi='DOI'; volumen='Volumen'; número='numero'
    páginas='Páginas'; semestre='Semestre'; idioma='Idioma'
    tipoRevista='Tipo de Revista'; Colciencias='Clasificación Colciencias'
    proyecto='Producto asociado a un proyecto de extensión o investigación'
    udea['Nombre del proyecto de extensión o investigación']=''
    autores='Autores'
    
udea[título]=udea['TI']
udea[revista]=udea['SO']
udea[issn]=udea['SN'].str.replace('\n','')
udea[doi]=udea['DI']
udea[volumen]=udea['VL'].str.replace('\n','')
udea[número]=udea['IS'].str.replace('\n','')
udea[páginas]=udea.apply(lambda row:
                  row['BP'].replace('\n','')+'-'+row['EP'].replace('\n','') 
                  if row['BP'] 
                  else row['BP'],axis='columns').str.replace('\-$','')
udea[año]=udea['PY']
udea[semestre]=None
udea[idioma]=udea['LA'].str.replace('\n','')
udea[tipoRevista]=udea.apply(lambda row: 'Revista Internacional Indexada' 
           if row.get('Tipo').find('WOS')>-1 or row.get('Tipo').find('SCP')
           else 'Revista Indexada en SCIELO',axis='columns')
udea[Colciencias]=None
udea[proyecto]=None


In [399]:
udea[autores]=udea['UDEA_authors'].apply(new_Autores)

In [400]:
udea_au=udea[udea[autores].apply(len)>0].reset_index(drop=True)

In [401]:
udea.PY.max()

2018

In [402]:
f=open('example.json','w')
f.write(
'''
[{
  "autores" : [ 
      {
          "nombreAutor" : "Jonathan Aguilar Bedoya",
          "identificacionAutor" : null,
          "tipoAutor" : "Estudiante de posgrado",
          "programaAcademico" : [ 
              "60178-MAESTRÍA EN INGENIERÍA MECÁNICA-INVESTIGACIÓN"
          ],
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ]
      }, 
      {
          "nombreAutor" : "Ainhoa Rubio Celemente",
          "tipoAutor" : "Autor externo",
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ],
          "institucion" : "Tecnológico de Antioquia. Institución Universitaria",
          "pais" : "Colombia"
      }, 
      {
          "nombreAutor" : "Laura Isable Velasquez Garcia",
          "tipoAutor" : "Profesor ocasional",
          "programaAcademico" : [ 
              "516-INGENIERÍA MECÁNICA"
          ],
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ]
      }, 
      {
          "nombreAutor" : "Edwin Lenin Chica Arrieta",
          "identificacionAutor" : "78761560",
          "tipoAutor" : "Profesor vinculado",
          "programaAcademico" : [ 
              "516-INGENIERÍA MECÁNICA"
          ],
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ]
      }
  ],
  "tituloArticulo" : "Design and Optimization of a Multi-Element Hydrofoil for a Horizontal-Axis Hydrokinetic Turbine",
  "nombreRevista" : "Energies",
  "issn" : "1996-1073",
  "doi" : "10.3390/en12244679",
  "volumen" : 12,
  "numero" : 24,
  "paginas" : "1-18",
  "annio" : 2019,
  "semestre" : "2019-2",
  "idioma" : "Inglés",
  "tipoRevista" : "Revista Internacional Indexada",
  "clasificacionColciencias" : "A1",
  "proyectoAsociado" : null
}
]
'''
)
f.close()

In [403]:
if JSON:
    new_format=pd.read_json( 'example.json' )
else:
    new_format=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vTiaxuZSGmI-aFgMUVjRAU3ws7WN9xmtjMEWu_SLOd5kAq_ZAuUtJUVr8qxNl3sMcp_fjE2gLVt_tdp/pub?output=xlsx')

In [404]:
audea=udea_au[list(new_format.columns)]
#.to_excel('UDEA_2018.xlsx',index=False)

In [405]:
audea.shape

(12270, 14)

# NOTA
Se ha añadido 
* `'Facultad'`
* y `'afiliacionWOS'` cuando no se conoce el departamente

In [418]:
def new_Autores(l,AU=AU,
                nombre='nombreAutor',#str 
                tipo='tipoAutor',
                identificación='identificacionAutor',
                facultad='facultad',#str
                departamento='departamento',#list
                grupo='grupoInvestigacion',#list
                institución='institucion'#str
               ):
    '''
    AU: Data Frame with author info
    Added Facultad y afiliacion WOS
    '''
    dd={}
    ll=[]
    for d in l:
        if d.get('CÉDULA'):
            au=AU[AU['CÉDULA']==int(d.get('CÉDULA'))].reset_index(drop=True)
            if not au.empty:
                dd[nombre]=au.loc[0,'NOMBRE COMPLETO']
                dd[tipo]='Profesor vinculado'
                dd[identificación]=au.loc[0,'CÉDULA']
                dd[facultad]=au.loc[0,'FACULTAD']
                dd[departamento]=[au.loc[0,'DEPARTAMENTO']]
                dd[grupo]=[au.loc[0,'GRUPO']]
                dd[institución]='Universidad de Antioquia'
                try:
                    dd['afiliacionWOS']=d.get('WOS_affiliation')[0]
                except:
                    pass
            else:
                if d.get('NOMBRE COMPLETO'):
                    dd[nombre]=d['NOMBRE COMPLETO']
                    dd[tipo]='Profesor vinculado'
                if d.get('CÉDULA'):
                    dd[identificación]=d['CÉDULA']
                    dd[institución]='Universidad de Antioquia'
                    try:
                        dd['afiliacionWOS']=d.get('WOS_affiliation')[0]
                    except:
                        pass
                else:
                    dd[identificación]=None
        ll.append(dd)
        dd={}
    return ll

In [421]:
udea[autores]=udea['UDEA_authors'].apply(new_Autores)

In [422]:
udea[autores].loc[5]

[{'afiliacionWOS': 'Univ Antioquia, Grp Corros & Protecc CIDEMAT, Medellin, Colombia.',
  'departamento': ['Departamento de Ingeniería Metalúrgica'],
  'facultad': 'Facultad de Ingeniería',
  'grupoInvestigacion': ['Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección'],
  'identificacionAutor': 74322987,
  'institucion': 'Universidad de Antioquia',
  'nombreAutor': 'Felix Echeverria Echeverria',
  'tipoAutor': 'Profesor vinculado'},
 {'afiliacionWOS': 'Univ Antioquia, Colombia.',
  'departamento': ['Departamento de Ingeniería Metalúrgica'],
  'facultad': 'Facultad de Ingeniería',
  'grupoInvestigacion': ['Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección'],
  'identificacionAutor': 71690172,
  'institucion': 'Universidad de Antioquia',
  'nombreAutor': 'Juan Guillermo Castaño Gonzalez',
  'tipoAutor': 'Profesor vinculado'},
 {'afiliacionWOS': 'Un

In [442]:
udea['Q']=0

In [438]:
import itertools
def add_and_check_wos_authors(row,autores='autores',nombre='nombreAutor'):
    wos_list =[ d.get('WOS_author')  for d in row.get('authors_WOS')]
    wos_aff  =[ d.get('affiliation') for d in row.get('authors_WOS')]

    udea_list=[ d.get('WOS_author') for d in row.get('UDEA_authors')]
    #Quality check: UdeA author in WOS_author
    print(udea_list)
    for uli in udea_list:
        print(uli)
        for ul in uli:
            ur=process.extractOne(ul, wos_list,
                                 scorer=fuzz.ratio)
            print(ul,ur)
            #If failed remove

        print('+++'*20)

    print('='*80)
    udea_list = list(itertools.chain.from_iterable(udea_list))

    #Complete missing authors
    for i in range( len(wos_list) ): 
        r=process.extractOne(wos_list[i], udea_list,
                                 scorer=fuzz.ratio)
            #if r and r[1]==100 → WOS_author is UDEA_author
        if r[1]<100:
            dd={}
            dd[nombre]=' '.join( wos_list[i].split(', ')[::-1] )
            dd['afiliacionWOS']=wos_aff[i][0]                
                #Else → Add to authors column
            row[autores].append(dd)
            print(wos_list[i],r)
    pass

In [439]:
i=5
udea.loc[[i]].apply(add_and_check_wos_authors, axis='columns')

[['Echeverria, Felix', 'Echeverria Echeverria, F.', 'Echeverria, F.', 'Echeverria Echeverria, Felix'], ['Castano Gonzalez, Juan Guillermo', 'Castano, J. G.', 'Castano, Juan Guillermo', 'Castano, Juan', 'Castano, Juan G.', 'Castano-Gonzalez, Juan', 'Castano-Gonzalez, J. G.', 'Guillermo Castano, Juan'], ['Gomez Botero, Maryory Astrid', 'Gomez, Maryory', 'GOMEZ, MARYORY', 'GOMEZ, MARYORY ASTRID', 'Gomez, Maryory Astrid', 'Gomez Botero, Maryory', 'Gomez, M.', 'Gomez, M. A.', 'Astrid Gomez-Botero, Maryory'], ['Velilla, Esteban', 'Velilla, E.']]
['Echeverria, Felix', 'Echeverria Echeverria, F.', 'Echeverria, F.', 'Echeverria Echeverria, Felix']
Echeverria, Felix ('Echeverria, Felix', 100)
Echeverria Echeverria, F. ('Echeverria, Felix', 63)
Echeverria, F. ('Echeverria, Felix', 87)
Echeverria Echeverria, Felix ('Echeverria, Felix', 76)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
['Castano Gonzalez, Juan Guillermo', 'Castano, J. G.', 'Castano, Juan Guillermo', 'Castano, Juan', 

5    None
dtype: object

In [440]:
udea[autores].loc[i]

[{'afiliacionWOS': 'Univ Antioquia, Grp Corros & Protecc CIDEMAT, Medellin, Colombia.',
  'departamento': ['Departamento de Ingeniería Metalúrgica'],
  'facultad': 'Facultad de Ingeniería',
  'grupoInvestigacion': ['Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección'],
  'identificacionAutor': 74322987,
  'institucion': 'Universidad de Antioquia',
  'nombreAutor': 'Felix Echeverria Echeverria',
  'tipoAutor': 'Profesor vinculado'},
 {'afiliacionWOS': 'Univ Antioquia, Colombia.',
  'departamento': ['Departamento de Ingeniería Metalúrgica'],
  'facultad': 'Facultad de Ingeniería',
  'grupoInvestigacion': ['Centro de Investigación, Innovación y Desarrollo de Materiales - CIDEMAT - Anteriormente: Grupo de Corrosión y Protección'],
  'identificacionAutor': 71690172,
  'institucion': 'Universidad de Antioquia',
  'nombreAutor': 'Juan Guillermo Castaño Gonzalez',
  'tipoAutor': 'Profesor vinculado'},
 {'afiliacionWOS': 'Un

In [343]:
wos_list=[ d.get('WOS_author') for d in udea['authors_WOS'].loc[i]]
wos_list


['Hoyos Barreto, Andres Emilio', 'Jimenez Correa, Monica Maria']

In [341]:
udea['authors_WOS'].loc[i]

[{'WOS_author': 'Hoyos Barreto, Andres Emilio',
  'affiliation': ['Univ Antioquia, Bogota, Colombia.'],
  'i': 0},
 {'WOS_author': 'Jimenez Correa, Monica Maria',
  'affiliation': ['Univ Antioquia, Bogota, Colombia.'],
  'i': 1}]

In [342]:
udea['UDEA_authors'].loc[i]

[{'CÉDULA': 32508663,
  'INICIALES': 'D. C.',
  'NOMBRE COMPLETO': 'Del Consuelo Montes Ramirez',
  'NOMBRES': 'Del Consuelo',
  'PRIMER APELLIDO': 'Montes',
  'SEGUNDO APELLIDO': 'Ramirez',
  'WOS_affiliation': ['SIU, Univ Antioquia, Environm Catalysis Res Grp, Medellin 1226, Colombia.',
   'Univ Antioquia, SIU, Environm Catalysis Res Grp, Medellin, Colombia.'],
  'WOS_author': ['Montes, Consuelo'],
  'full_name': 'MONTES RAMIREZ MARIA DEL CONSUELO'}]

In [213]:
aa[aa.apply(

(12270,)

In [175]:
udea['authors_WOS']

0                                                                                    [{'i': 0, 'WOS_author': 'Garcia Quintero, Edwin', 'affiliation': ['Univ Antioquia, Dept Ingn Elect, Medellin, Colombia.']}]
1        [{'i': 0, 'WOS_author': 'Hoyos Barreto, Andres Emilio', 'affiliation': ['Univ Antioquia, Bogota, Colombia.']}, {'i': 1, 'WOS_author': 'Jimenez Correa, Monica Maria', 'affiliation': ['Univ Antioqui...
2        [{'i': 0, 'WOS_author': 'Soler T, Wilmer', 'affiliation': ['Univ Antioquia, Fac Med, Medellin 1226, Colombia.']}, {'i': 1, 'WOS_author': 'Velasquez E, Nelly D.', 'affiliation': ['Univ Antioquia, F...
3                                                                                                                                                                                                             []
4        [{'i': 0, 'WOS_author': 'Ballestas Cueto, H.', 'affiliation': ['Grupo de investigación Salud y Sostenibilidad, Escuela de Microbiología, Univ Antioquia, Me

In [157]:
au=AU[AU['CÉDULA']==int(618927154.0)].reset_index(drop=True)
au.empty

True

In [159]:
udea_au[list(new_format.columns)]

False

In [149]:
audea.loc[1].to_dict()

{'annio': 2008,
 'autores': [{'Identificación': 32508663,
   'Nombres': 'Del Consuelo Montes Ramirez',
   'Tipo': 'Profesor vinculado'}],
 'clasificacionColciencias': None,
 'doi': '',
 'idioma': 'Spanish',
 'issn': '0120-5609',
 'nombreRevista': 'INGENIERIA E INVESTIGACION',
 'numero': '3',
 'paginas': '41-46',
 'proyectoAsociado': None,
 'semestre': None,
 'tipoRevista': 'Revista Internacional Indexada',
 'tituloArticulo': 'Cement plant gaseous pollutant emission reduction technologies',
 'volumen': '28'}

In [69]:
exl = [{'id': 123, 'data': 'qwerty', 'indices': [1,10]}, {'id': 345, 'data': 'mnbvc', 'indices': [2,11]}]

In [70]:
with open("kk.json", "w") as outfile: 
    json.dump(exl, outfile)

In [75]:
ex.to_json('kk.json',orient='records')

In [76]:
cat kk.json

[{"annio":2019,"autores":[{"nombreAutor":"Jonathan Aguilar Bedoya","programaAcademico":["60178-MAESTR\u00cdA EN INGENIER\u00cdA MEC\u00c1NICA-INVESTIGACI\u00d3N"],"identificacionAutor":null,"grupoInvestigacion":["Grupo de Energ\u00eda Alternativa"],"tipoAutor":"Estudiante de posgrado"},{"nombreAutor":"Ainhoa Rubio Celemente","institucion":"Tecnol\u00f3gico de Antioquia. Instituci\u00f3n Universitaria","pais":"Colombia","grupoInvestigacion":["Grupo de Energ\u00eda Alternativa"],"tipoAutor":"Autor externo"},{"nombreAutor":"Laura Isable Velasquez Garcia","programaAcademico":["516-INGENIER\u00cdA MEC\u00c1NICA"],"grupoInvestigacion":["Grupo de Energ\u00eda Alternativa"],"tipoAutor":"Profesor ocasional"},{"nombreAutor":"Edwin Lenin Chica Arrieta","programaAcademico":["516-INGENIER\u00cdA MEC\u00c1NICA"],"identificacionAutor":"78761560","grupoInvestigacion":["Grupo de Energ\u00eda Alternativa"],"tipoAutor":"Profesor vinculado"}],"clasificacionColciencias":"A1","doi":"10.3390\/en12244679","id

In [77]:
pd.read_json('kk.json')

,annio,autores,clasificacionColciencias,doi,idioma,issn,nombreRevista,numero,paginas,proyectoAsociado,semestre,tipoRevista,tituloArticulo,volumen
0,2019,"[{'nombreAutor': 'Jonathan Aguilar Bedoya', 'programaAcademico': ['60178-MAESTRÍA EN INGENIERÍA MECÁNICA-INVESTIGACIÓN'], 'tipoAutor': 'Estudiante de posgrado', 'identificacionAutor': None, 'grupo...",A1,10.3390/en12244679,Inglés,1996-1073,Energies,24,1-18,NaN,2019-2,Revista Internacional Indexada,Design and Optimization of a Multi-Element Hydrofoil for a Horizontal-Axis Hydrokinetic Turbine,12


In [78]:
with open(r"kk.json", "r") as read_file:
    data = json.load(read_file)

In [79]:
data

[{'annio': 2019,
  'autores': [{'grupoInvestigacion': ['Grupo de Energía Alternativa'],
    'identificacionAutor': None,
    'nombreAutor': 'Jonathan Aguilar Bedoya',
    'programaAcademico': ['60178-MAESTRÍA EN INGENIERÍA MECÁNICA-INVESTIGACIÓN'],
    'tipoAutor': 'Estudiante de posgrado'},
   {'grupoInvestigacion': ['Grupo de Energía Alternativa'],
    'institucion': 'Tecnológico de Antioquia. Institución Universitaria',
    'nombreAutor': 'Ainhoa Rubio Celemente',
    'pais': 'Colombia',
    'tipoAutor': 'Autor externo'},
   {'grupoInvestigacion': ['Grupo de Energía Alternativa'],
    'nombreAutor': 'Laura Isable Velasquez Garcia',
    'programaAcademico': ['516-INGENIERÍA MECÁNICA'],
    'tipoAutor': 'Profesor ocasional'},
   {'grupoInvestigacion': ['Grupo de Energía Alternativa'],
    'identificacionAutor': '78761560',
    'nombreAutor': 'Edwin Lenin Chica Arrieta',
    'programaAcademico': ['516-INGENIERÍA MECÁNICA'],
    'tipoAutor': 'Profesor vinculado'}],
  'clasificacionColci

In [80]:
with open("kkk.json", "w") as outfile: 
    json.dump(data, outfile) 

In [81]:
cat kkk.json

[{"doi": "10.3390/en12244679", "tipoRevista": "Revista Internacional Indexada", "numero": 24, "semestre": "2019-2", "annio": 2019, "idioma": "Ingl\u00e9s", "nombreRevista": "Energies", "issn": "1996-1073", "paginas": "1-18", "proyectoAsociado": null, "volumen": 12, "autores": [{"nombreAutor": "Jonathan Aguilar Bedoya", "programaAcademico": ["60178-MAESTR\u00cdA EN INGENIER\u00cdA MEC\u00c1NICA-INVESTIGACI\u00d3N"], "tipoAutor": "Estudiante de posgrado", "identificacionAutor": null, "grupoInvestigacion": ["Grupo de Energ\u00eda Alternativa"]}, {"nombreAutor": "Ainhoa Rubio Celemente", "pais": "Colombia", "tipoAutor": "Autor externo", "grupoInvestigacion": ["Grupo de Energ\u00eda Alternativa"], "institucion": "Tecnol\u00f3gico de Antioquia. Instituci\u00f3n Universitaria"}, {"nombreAutor": "Laura Isable Velasquez Garcia", "programaAcademico": ["516-INGENIER\u00cdA MEC\u00c1NICA"], "grupoInvestigacion": ["Grupo de Energ\u00eda Alternativa"], "tipoAutor": "Profesor ocasional"}, {"nombreAut